In [1]:
import os
import pandas as pd
import ftplib
import SimpleITK as sitk

In [2]:
def getFile(ftp, base_path, filename):
    try:
        ftp.retrbinary("RETR " + filename ,open(os.path.join(base_path,filename), 'wb').write)
    except:
        print("Error")

In [3]:
output_directory = r'C:\Users\LydiaN\OneDrive - Personal\OneDrive\Work\DigitalAssets\BICCNMorphologyInventory\u19zeng'
inventory_path = os.path.join(output_directory,'sample_inventory_with_image_info.csv')
inventory = pd.read_csv(inventory_path)
inventory.set_index('Sample ID', inplace=True)

In [4]:
def get_dir_info(ftp) :
    
    data = []
    ftp.dir(data.append)
    data = [x.split() for x in data if x != '']
    data = pd.DataFrame(data)
    
    return data

def check_for_directory(data,dirname) :
    
    has_dir = False
    for dindex, drow in data.iterrows() :
        if drow[8] == dirname :
            has_dir = True
    return has_dir

In [5]:
# come back and do 'mouseID_339951-17781'
done_list = inventory.index[0:9].values
print(done_list)

['mouseID_210254-15257' 'mouseID_236174-16124' 'mouseID_297974-17109'
 'mouseID_314107-17300' 'mouseID_321237-17302' 'mouseID_321244-17545'
 'mouseID_325875-17543' 'mouseID_326952-17304' 'mouseID_339951-17781']


In [6]:
request_list = ['mouseID_339951-17781']
print(request_list)

In [7]:
count = 0
for iindex, irow in inventory.iterrows() :
    
    if iindex not in request_list :
        continue
    
    ftp = ftplib.FTP(host="download.brainimagelibrary.org")
    ftp.login()

    home_dir = ftp.pwd()
    remote_base = "biccn/zeng/luo/fMOST"
    
    print(iindex)
    
    # specimen output directory
    spath = os.path.join(output_directory,iindex)
    
    # get listing of remote directory
    rpath = remote_base + "/" + iindex
    ftp.cwd( rpath )
    data = get_dir_info( ftp )
    
    dlist = ['green','raw','fullres']
    
    for d in dlist :
        data = get_dir_info(ftp)
        has_dir = check_for_directory( data, d )
        if not has_dir :
            break
        ftp.cwd(d)
        
    if d != dlist[-1] :
        print("No raw/fullres directory - skipping")
        break
        continue        
        
    print("start dir")
    data = get_dir_info(ftp)
    print("end dir")
    
    matched = ['tif' in x for x in data[8]]
    filtered = data[matched]
    ordered = filtered.sort_values(by=8)
    first_file = ordered.loc[ordered.index[0], 8]
    last_file = ordered.loc[ordered.index[-1], 8]
    mid_file = ordered.loc[ordered.index[5000], 8]
    print(first_file, mid_file, last_file)
    
    tpath = os.path.join( spath, 'green', 'raw', 'fullres')
    os.makedirs(tpath,exist_ok=True)
    
    print("start download")
    getFile(ftp, tpath, mid_file )
    print("end download")
        
    inventory.loc[iindex,'first_full_green_image'] = first_file
    inventory.loc[iindex,'last_full_green_image'] = last_file
    inventory.loc[iindex,'example_full_green_image'] = mid_file
    
    ftp.quit()


mouseID_339951-17781
start dir
end dir
test_00011_mon.tif test_05011_mon.tif test_11031_mon.tif
start download
end download
